# Discrete Laplacian

This notebook is the solution and implementation to some problems in (Crane, 2018).

Let $M$ be closed compact and oriented surface in $\mathbb{R}^3$. Let $K$ be a triangulation of $M$, such that every angle is acute; and enumerate the vertex set of $K$ with $\lbrace 0, \ldots, \lvert{K}\rvert\rbrace$.

The Laplace-Beltrami $-\Delta$ is given as
$$
    -\Delta u = -\nabla \cdot (\nabla u) \colon H^1 \to H^{-1}
$$
where $\nabla\cdot$ and $\nabla$ are the divergence and gradient operator over the physical domain, respectively. $H^1$ denote the space of $L^2$-weak differentiable functions over the domain and $H^{-1}$ its dual.

## A formulation of $\Delta$ using FEM

Let $N_j$ be the hat function that gets the value of $1$ at node/vertex $j$ and $0$ on its neighbors. 
The action of the laplacian on every $N_j$ gives us the value of the laplacian at node $j$. Therefore, we are interested in the vector
$$
    \mathbf{U} 
    = 
    (\langle \Delta u, N_0 \rangle, \langle \Delta u, N_1 \rangle, \ldots, \langle \Delta u, N_{\lvert K\rvert} \rangle).
$$
Using integration by parts and the divergence theorem in each element we get:
$$
\begin{aligned}
    \langle \Delta u, N_j \rangle 
    &=
    \sum_{T\colon j\in T} \langle \Delta u, N_j \rangle_{T}
    \\&=
    \sum_{T\colon j\in T} \langle \nabla u, \nabla N_j\rangle + \langle \mathbf{n} \cdot \nabla u, N_j\rangle_{\partial T},
\end{aligned}
$$
where $\mathbf{n}$ is the outward normal of the triangle involve, and the sum runs for all triangles in the mesh. Observe that
$$
    \sum_{T\colon j\in T} \langle \mathbf{n} \cdot \nabla u, N_j\rangle_{\partial T}
    =
    \sum_{e\in\text{Edges}} \langle \mathbf{n}\cdot\nabla u, N_j\rangle_{e} - \langle \mathbf{n}\cdot\nabla u, N_j\rangle_{e}
    = 0.
$$
All together, the action of the Laplace-Beltrami operator reads:
$$
    \langle \Delta u, N_j \rangle = \sum_{T\colon j\in T} \langle \nabla u, \nabla N_j\rangle_{T}.
$$
Discretizing $u = \sum_{i=0}^{\lvert{K}\rvert} u_i N_i$ we get that:
$$
    \langle \nabla u, \nabla N_j\rangle_{T}
    =
    \sum_{i=0}^{\lvert K \rvert} u_i \langle \nabla N_i, \nabla N_j\rangle_{T}
    =
    \sum_{i\in T} u_i \langle \nabla N_i, \nabla N_j \rangle_T.
$$
Therefore, the action of the Laplace-Beltrami operator on each shape function reads:
$$
    \langle \Delta u, N_j \rangle
    =
    \sum_{T\colon j\in T} \langle \nabla u, \nabla N_j\rangle_{T}
    =
    \sum_{T\colon j\in T} \sum_{i\in T} u_i \langle \nabla N_i, \nabla N_j\rangle_{T}.
$$
This sum runs for all triangles and all vertex in those triangles. We can rearrange it by taking the sum over all neighbors $i$ of $j$ and compute the action for the two triangles adyacent to the edge $ij$; for the case $i=j$ we need to account for every triangle containing $j$ and compute the action there. In all, we have the following expression.
$$
    \langle \Delta u, N_j \rangle
    =
    \sum_{i\in N(j)} u_i 
        \left(
        \langle \nabla N_i, \nabla N_j\rangle_{T_i}
        + 
        \langle \nabla N_i, \nabla N_j\rangle_{T_i'}
        \right)
    +
    \sum_{T\colon j\in T}
        u_j \langle \nabla N_j, \nabla N_j\rangle_{T}.
$$
Summarizing:
$$
    \mathbf{U}_{j} =
        \sum_{i\in N(j)} 
        \left(
        \langle \nabla N_i, \nabla N_j\rangle_{T_i}
        + 
        \langle \nabla N_i, \nabla N_j\rangle_{T_i'}
        \right)
        +
        \sum_{T\colon j\in T}
        \langle \nabla N_j, \nabla N_j\rangle_{T}.
$$

## Computing $\mathbf{U}$

In order to compute $\mathbf{U}$, we need to compute two kind of inner products:

1. $\langle \nabla N_i, \nabla N_j \rangle_{T}$ for $i\ne j$
2. $\langle \nabla N_i, \nabla N_i \rangle_{T}$.

Let's start by computing $\nabla N_i$, since we need it for all computations. Let $e$ be the edge opposite to $i$ in the triangle 
$T$. Since $N_i$ is linear, its gradient is a constant vector. Given that $N_i$ vanishes over $e$ and increases linearly until the value $1$ at node $i$, we may conclude that
$$
    \nabla N_i = - \frac{1}{h} \mathbf{n}_i,
$$
where $\mathbf{n}_i$ is the outward unit normal to $T$ at $e$ and $h$ is the height of the triangle measured from $e$ to $i$.
Let $b$ denote the length of $e$. Then, $|T| = \frac{1}{2} hb$. Solving for $h$ and substituting in the equation above yields:
$$
    \nabla N_i = -\frac{b}{2|T|} \mathbf{n}_i.
$$

**1.** Let $i$ and $j$ be neighboring vertex. Then, 
$$
    (\nabla N_i, \nabla N_j)
    =
    \frac{b_i b_j}{4 |T|^2} \int_{T} \mathbf{n}_i \cdot \mathbf{n}_j
    =
    - \frac{b_i b_i}{4 |T|} \cos(\theta_{ij}),
$$
where $\theta_{ij}$ is the angle between $\mathbf{n}_i$ and $\mathbf{n}_j$. The minus sign is a consequence of the acute angle requirement (the reader is invited to draw). Given that $2|T| = b_i b_j\sin(\theta_{ij})$ the expression simplifies to:
$$
    \langle \nabla N_i, \nabla N_j \rangle_{T}
    =
    -\frac{1}{2} \cot(\theta_{ij}).
$$

**2.** Let's compute it directly, with the same notation as above:
$$
    (\nabla N_i, \nabla N_i)
    =
    \frac{b^2}{4 |T|} |T|
    =
    \frac{b}{2 h}
    =
    \frac{1}{2} (\cot \alpha_i + \cot \alpha_j),
$$
where the last inequality follows from $2|T| = bh$ and the identity $b/h = \cot\alpha_i + \cot\beta_i$. Here's a short proof of the identity: Split $T$ into two right triangles. Let $b_1$ and $b_2$ denote the diagonals associated to angles $\alpha_i$ and $\beta_i$, respectively. Then
$$
    \cot\alpha_i + \cot\beta_i
    = \frac{b_1 \cos\alpha_i}{h} + \frac{b_2\cos\beta_i}{h} 
    = \frac{b}{h}.
$$

Let $N(j)$ denote the neighbors of $j$. Now the expression for $\mathbf{U}$ reads
$$
\begin{aligned}
    \mathbf{U}_{j} 
    &= 
    \sum_{i\in N(j)} u_i
    \left(
        \frac{1}{2} \cot(\theta_{ij})
        +
        \frac{1}{2} \cot(\theta_{ij}')
    \right)
    -
    \sum_{T\colon j\in T} u_j
        \frac{1}{2} (\cot_j \alpha + \cot \beta_j)
    \\&=
    \frac{1}{2} \left(
        \sum_{i\in N(j)} u_i 
        \left( \cot(\theta_{ij}) + \cot(\theta_{ij}')\right)
    -
    u_j \sum_{\theta \textrm{ opposite to } j} \cot\theta
    \right)
    \\&=
    \frac{1}{2} \left(
        \sum_{i\in N(j)} u_i 
        \left( \cot(\theta_{ij}) + \cot(\theta_{ij}')\right)
    -
    u_j \sum_{i\in N(j)} \left( \cot(\theta_{ij}) + \cot(\theta_{ij}')\right)
    \right)
    \\&=
    \frac{1}{2} \sum_{i\in N(j)} \left( \cot(\theta_{ij}) + \cot(\theta_{ij}')\right) (u_i - u_j)
\end{aligned}
$$
where $\theta_{ij}$ and $\theta_{ij}'$ in each sum being the angle opposite to edge $ij$ in each triangle.

We have arrived to the famous cotan formula.

## Matrix form

In order to perform computations effiently, we'll rewrite $\mathbf{U} = \mathbf{L}\vec{u}$, for a matrix $\mathbf{L}$. So that $\mathbf{L} \approx \Delta$.

From the equation defining $\mathbf{U}$ we get that
$$
    2 \mathbf{L}_{ij}
    =
    \begin{cases}
        \cot(\theta_{ij}) + \cot(\theta_{ij}') &, \textrm{ if } i\ne j\\
        -\sum_{k=0, k\ne j}^{\lvert{K}\rvert} L_{ik} &, \textrm{ if } i=j
    \end{cases}.
$$

### Extra: Computing $\cot$

Let $x$ and $y$ be the vectors whose angle is $\theta$. Then, $\frac{x\cdot y}{\|x\times y\|} = \cot(\theta)$.

In [1]:
from scipy.sparse import lil_matrix
import numpy as np

def laplace_matrix(nodes: list, triangles: list) -> (lil_matrix, lil_matrix):
    '''
    Input:
        - nodes : [#nodes]x[3]. nodes[i] contains the cartesian coordinates of the ith node.
        - triangles :[#triangles]x[3]. triangles[i] contains the indexes of the vertices of the ith triangle.
    Output:
        - the cotan-Laplace matrix in lil_matrix format.
        - the mass matrix
    '''
    N = len(nodes)
    L = lil_matrix((N,N), dtype=np.float32)
    M = lil_matrix((N,N), dtype=np.float32)
    
    # Fill non diagonals
    for T in triangles:
        # Fill cotan matrix
        i, j, k = T
        x, y, z = nodes[T]
        
        # Relative vectors
        ie1 = y-x; ie2 = z-x
        je1 = z-y; je2 = x-y
        ke1 = x-z; ke2 = y-z

        # Mass matrix
        area = np.linalg.norm(np.cross(ie1, ie2))
        M[i,i] += area
        M[k,k] += area
        M[j,j] += area

        # Compute cotangents
        cot_i = abs(np.dot(ie1, ie2)) / np.linalg.norm(np.cross(ie1, ie2))
        cot_j = abs(np.dot(je1, je2)) / np.linalg.norm(np.cross(je1, je2))
        cot_k = abs(np.dot(ke1, ke2)) / np.linalg.norm(np.cross(ke1, ke2))
        
        L[i,j] += cot_k; L[j,i] += cot_k
        L[i,k] += cot_j; L[k,i] += cot_j
        L[j,k] += cot_i; L[k,j] += cot_i
    
    # Fill diagonals
    for i in range(N):
        L[i, i] = -sum([L[i,j] for j in range(N)])
    
    return L/2, M/6

# Example

In [2]:
import pyvista as pv
pv.set_jupyter_backend('trame')

import numpy as np
from scipy.sparse import lil_matrix, identity
import scipy.sparse.linalg as spla

import matplotlib.pyplot as plt

In [3]:
def save(filename, mesh, **kwargs):
    pl = pv.Plotter()
    pl.add_mesh(mesh, **kwargs)
    pl.save_graphic(filename)
    return pl

In [4]:
MAX_FACES = 15_000

#  Mesh from https://www.thingiverse.com/thing:6703649/files
mesh = pv.PolyData('resources/3D_models/blue_whale/files/ballena_azul_Lowpoly.stl')

num_faces = len(mesh.faces) // 4

if num_faces > MAX_FACES:
    mesh = mesh.decimate(1.0 - (MAX_FACES / num_faces))
    print('Decimated mesh, from ', num_faces, 'faces to', len(mesh.faces)//4, 'faces')

nodes = mesh.points
triangles = np.delete(mesh.faces.reshape(-1, 4), 0, 1)

print('# Nodes  :', nodes.shape)
print('# Faces  :', triangles.shape)

# Nodes  : (1845, 3)
# Faces  : (3686, 3)


In [5]:
save('images/discrete_laplacian_mesh.svg', mesh);

<img src="images/discrete_laplacian_mesh.svg"></img>

In [6]:
L, M = laplace_matrix(nodes, triangles)

In [7]:
from random import sample

default_cmap = plt.get_cmap('seismic')

source_indices = [np.argmax(nodes[:, 0]), np.argmax(nodes[:, 1] + nodes[:, 0])]
source_field = np.zeros(len(nodes))
source_field[source_indices] = [(-1)**i/len(source_indices) for i in range(len(source_indices))]

vmax = np.max(source_field)
vmin = np.min(source_field)

In [8]:
save('images/discrete_laplacian_heat_source.svg', mesh, scalars=source_field, clim=[vmin, vmax], cmap=default_cmap);
#  mesh.plot(scalars=source_field, clim=[vmin, vmax], cmap=default_cmap)

<img src="images/discrete_laplacian_heat_source.svg"></img>

In [9]:
steady_heat = -spla.spsolve(L.tocsr(), source_field)

In [10]:
save('images/discrete_laplacian_steady_heat.svg', mesh, scalars=steady_heat, cmap=default_cmap, show_scalar_bar=True);
# mesh.plot(scalars=steady_heat, cmap=default_cmap, show_scalar_bar=True)

<img src="images/discrete_laplacian_steady_heat.svg"></img>

# References

* Crane, K., 2018. Discrete differential geometry: An applied introduction. Notices of the AMS, Communication, 1153. Available at: https://www.cs.cmu.edu/~kmcrane/Projects/DDG/ (Accessed: 9 May 2025).